##### Copyright 2020 Google LLC.
Licensed under the Apache License, Version 2.0 (the "License");

In [0]:
# Copyright 2020 The Google Research Authors.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# M-layer Robustness

This notebook analyzes model robustness and validates robustness claims stated in "Intelligent Matrix Exponentiation".

In [0]:
# Imports and set-up.

import collections
import contextlib
import hashlib
import math
import numbers
import operator
import os

from matplotlib import pyplot
import numpy
import opt_einsum  # numpy.einsum() cannot handle fancy-type arrays, this can.
import scipy.special

import logging
logging.getLogger('tensorflow').disabled = True

import tensorflow.compat.v1.keras as keras
import tensorflow_datasets as tfds

Download the `cifar10_model.npy` if not already present and check its integrity. 

In [0]:
if not os.path.isfile('cifar10_model.npy'):
  !wget https://raw.githubusercontent.com/google-research/google-research/master/m_layer/cifar10_model.npy
with open('cifar10_model.npy', 'rb') as h:
        fp = hashlib.sha256(h.read()).hexdigest()
        assert fp == ('abe15f55bffd5f26df664c87bd9a4db2',
                      'd2cbc05dd086bc39c860cc66b7dc1d25'),
                      "Corrupted model file detected."

In [0]:
# Auxiliary Affine Arithmetic Python code (basic AA implementation).

# Stack of dynamic "environmentally active" AAContext contexts.
# See AANum.__repr__ for an explanation.
_DynamicAAContexts = []  # pylint:disable=invalid-name


def _merge_ess(linop, ess1, ess2):
  """Merges weighted-error-symbol collections."""
  ess = {}
  for sym in set(ess1) | set(ess2):
    coeff = linop(ess1.get(sym, 0), ess2.get(sym, 0))
    if coeff:
      ess[sym] = coeff
  return ess


class AAContext(contextlib.AbstractContextManager):
  """Affine Arithmetic Context.

  Every AANumber refers to a context, which takes care of managing error
  symbols. Only numbers from the same AAContext can be combined.
  """

  def __init__(self, name=None,
               collapsing_threshold=0,
               thorough=True,
               max_num_symbols=float('inf')):
    """Initializes the instance.

    Args:
      name: The name of the context, for debugging.
      collapsing_threshold: Error symbols smaller than
        {largest error symbol} * {this factor} will get collected
        into a new error symbol.
      thorough: If True, work hard to properly keep track of products
        of error symbols. If False, only do this for the 1st level
        of products.
      max_num_symbols: The maximal number of error symbols to keep
        on one quantity.
    """
    # Error symbol expansions.
    # Key = Error symbol number.
    # Value = `True` for fundamental error-symbols, or a sorted tuple of
    # (symbol_id, power) of fundamental symbols and their power that
    # the symbol was obtained from.
    self._esym_expansions = {}
    # The reverse mapping for higher-order error-symbols, to find an
    # already-known error symbol given its expansion.
    self._esym_by_expansion = {}
    self._collapsing_threshold = float(collapsing_threshold)
    self._max_num_symbols = float(max_num_symbols)
    self._name = name
    self._thorough = thorough
    self._num_syms = 0
    self._num_mul = 0

  def __repr__(self):
    return '<AAContext name=%s>' % self._name

  def __enter__(self):
    _DynamicAAContexts.append(self)
    return self

  def __exit__(self, exc_type, exc_value, traceback):
    del exc_type, exc_value, traceback  # Unused.
    _DynamicAAContexts.pop()

  def product_symbol(self, sym_x, sym_y):
    """Finds an error symbol for the product of two error symbols."""
    expansion_x = self._esym_expansions.get(sym_x, False)
    expansion_y = self._esym_expansions.get(sym_y, False)
    if not self._thorough and not expansion_x is expansion_y is True:
      # Non-thorough operation, higher product symbol.
      ret = self._num_syms
      self._num_syms += 1
      return ret
    #
    if isinstance(expansion_x, bool):
      expansion_x = ((sym_x, 1),)
    if isinstance(expansion_y, bool):
      expansion_y = ((sym_y, 1),)
    accum_xy = {sym: power for sym, power in expansion_x}
    for esym_y, power in expansion_y:
      accum_xy[esym_y] = accum_xy.get(esym_y, 0) + power
    expansion_xy = tuple(sorted(accum_xy.items()))
    sym_xy = self._esym_by_expansion.get(expansion_xy)
    if sym_xy is not None:
      return sym_xy
    # Otherwise, make a new entry.
    sym_xy = self._num_syms
    self._num_syms += 1
    self._esym_expansions[sym_xy] = expansion_xy
    self._esym_by_expansion[expansion_xy] = sym_xy
    return sym_xy

  def new_symbol(self, fundamental=False):
    """Generates a new independent error symbol, e.g. for collapsing."""
    sym = self._num_syms
    self._num_syms += 1
    if fundamental:
      self._esym_expansions[sym] = True
    return sym

  def collapse_ess(self, ess):
    """Modifies ess by collapsing symbols with 'small' coefficients into one."""
    collapsing_threshold = self._collapsing_threshold
    if not ess:
      return  # Short-cut.
    if collapsing_threshold > 0:
      threshold_abs_coeff = collapsing_threshold * max(
          abs(x) for x in ess.values())
      to_collapse = {sym: x for sym, x in ess.items()
                     if abs(x) < threshold_abs_coeff}
    else:
      to_collapse = set()
    if len(ess) - len(to_collapse) + 1 > self._max_num_symbols:
      # Collapsing the symbols listed above will not get us to an acceptable
      # number of symbols.
      syms_by_coeff_magnitude = sorted(
          (sym_x for sym_x in ess.items() if sym_x[0] not in to_collapse),
          key=lambda sym_x: abs(sym_x[1]))
      for sym, c in syms_by_coeff_magnitude[:-int(self._max_num_symbols)]:
        to_collapse[sym] = c
    if to_collapse:
      collapsed_coeff = sum(map(abs, to_collapse.values()))
      for sym in to_collapse:
        del ess[sym]  # Removed by collapsing.
      ess[self.new_symbol()] = collapsed_coeff

  def num(self, val, radius=0):
    """Produces a new AANum with fresh error-symbol."""
    esym = self.new_symbol(fundamental=True)
    return AANum(val, (esym, radius), context=self)


class AANum(numbers.Number):
  """Affine-Arithmetic Number."""

  def __init__(self, val, *seq_esym_escale, context=None):
    """Initializes the instance."""
    if context is None:
      if not _DynamicAAContexts:
        raise RuntimeError('No AAContext available for initializing AANum.')
      context = _DynamicAAContexts[-1]
    self._context = context
    self._val = val
    self._ess = {esym: escale for esym, escale in seq_esym_escale}

  @property
  def radius(self):
    return sum(map(abs, self._ess.values()))

  def __str__(self):
    return '<AA %g +/- %g>' % (self._val, self.radius)

  def __repr__(self):
    # Implementing __repr__ is tricky, since we cannot also serialize
    # the context: The context must be identical for all the different
    # AANum instances in an arithmetic expression.
    # We resolve this by making __repr__ produce a representation that refers
    # to the 'top dynamic context' which is set by using AAContext as
    # a context-manager.
    return 'AANum(%r, %s)' % (
        self._val,
        ', '.join(map(repr, sorted(self._ess.items()))))

  def _linop(self, linop, other):
    """Lifts a linear operator to Affine Arithmetic."""
    if isinstance(other, AANum):
      if self._context is not other._context:
        raise ValueError('Cannot combine AANums from different contexts.')
      result = AANum(linop(self._val, other._val), context=self._context)
      ess = _merge_ess(linop, self._ess, other._ess)
      self._context.collapse_ess(ess)
      result._ess = ess
      return result
    else:
      # Case: AANum +- {non-AA number}.
      result = AANum(linop(self._val, other), context=self._context)
      result._ess = _merge_ess(linop, self._ess, {})
      return result

  def __add__(self, other):
    return self._linop(operator.add, other)

  def __radd__(self, other):
    return self.__add__(other)

  def __sub__(self, other):
    return self._linop(operator.sub, other)

  def __rsub__(self, other):
    return self._linop(lambda x, y: y - x, other)

  def __mul__(self, other):
    context = self._context
    context._num_mul += 1
    if not isinstance(other, AANum):
      # Case: AANum * {non-AA number}.
      result = AANum(self._val * other, context=context)
      if other != 0:
        result._ess = {sym: x * other for sym, x in self._ess.items()}
      return result
    # Otherwise, `other` is also an AANum.
    if context is not other._context:
      raise ValueError('Cannot multiply AANums from different contexts.')
    xval = self._val
    yval = other._val
    # Scaling error symbol coefficients with xval and yval.
    ess = {sym: x * yval for sym, x in self._ess.items()}
    for sym, y in other._ess.items():
      ess[sym] = ess.get(sym, 0) + xval * y
    # Adding error symbol coefficients for products of error-symbols.
    for sym_x, coeff_x in self._ess.items():
      for sym_y, coeff_y in other._ess.items():
        sym_xy = context.product_symbol(sym_x, sym_y)
        ess[sym_xy] = ess.get(sym_xy, 0) + coeff_x * coeff_y
    # Pruning
    for sym_to_delete in [sym for sym, x in ess.items() if x == 0]:
      del ess[sym_to_delete]  # Remove coefficient-zero entries.
    # Collapsing small-coefficient symbols into one.
    context.collapse_ess(ess)
    result = AANum(xval * yval, context=context)
    result._ess = ess
    return result

  def __rmul__(self, other):
    return self.__mul__(other)

  def __truediv__(self, other):
    if not isinstance(other, AANum):
      # Case: AANum / {non-AA number}.
      result = AANum(self._val / other, context=self._context)
      if other != 0:
        result._ess = {sym: x / other for sym, x in self._ess.items()}
      return result
    # Dividing an AANum by another AANum is not implemented yet.
    return NotImplemented

  def __float__(self):
    return float(self._val)


In [0]:
# Checking robustness claims from and around "Intelligent Matrix Exponentiation"

NUM_CLASSES = 10


def plot_loghist(xs, bins, filename=None, show=False):
  """Plots a histogram with logarithmic x-axis."""
  _, bins = numpy.histogram(xs, bins=bins)
  log_bins = numpy.logspace(numpy.log10(bins[0]),
                            numpy.log10(bins[-1]),
                            len(bins))
  fig = pyplot.figure()
  axes = fig.gca()
  axes.hist(xs, bins=log_bins, histtype='step')
  axes.set_xscale('log')
  axes.grid()
  axes.set_title('Certified-Robustness Bounds Distribution')
  if filename is not None:
    fig.savefig(filename)
  if show or filename is None:
    pyplot.show()
    fig.show()
  return fig


def sorted_xys(xs, ys):
  """Sorts features/labels into well-defined order.

  tfds.load('cifar10') does not give us a guaranteed order of examples.
  """
  fp_weights = numpy.random.RandomState(seed=0).uniform(
      size=xs.size // xs.shape[0])
  xys = sorted(zip(xs, ys),
               key=lambda xy: numpy.dot(fp_weights, xy[0].reshape(-1)))
  return (numpy.stack([x for x, _ in xys], axis=0),
          numpy.stack([y for _, y in xys], axis=0))


def load_cifar10():
  """Loads the CIFAR-10 dataset."""
  train = tfds.load('cifar10', split='train', with_info=False, batch_size=-1)
  test = tfds.load('cifar10', split='test', with_info=False, batch_size=-1)
  train_np = tfds.as_numpy(train)
  test_np = tfds.as_numpy(test)
  x_train, y_train = sorted_xys(train_np['image'], train_np['label'])
  x_test, y_test = sorted_xys(test_np['image'], test_np['label'])
  print(f'x_train shape: {x_train.shape}, x_test shape: {x_test.shape}')
  y_train_cat = keras.utils.to_categorical(y_train, NUM_CLASSES)
  y_test_cat = keras.utils.to_categorical(y_test, NUM_CLASSES)
  x_train_range1 = x_train.astype('float32') / 255
  x_test_range1 = x_test.astype('float32') / 255
  return ((x_train_range1, y_train_cat), (x_test_range1, y_test_cat))


def load_the_cifar10_model(filename='cifar10_model.npy'):
  return np_load_arrays_from_file(
      [('mb', (20, 20)), ('mk', (35, 20, 20)),
       ('sb', (10,)), ('sk', (20, 20, 10)),
       ('ub', (35,)), ('uk', (32, 32, 3, 35))],
      filename)


def _get_taylor_strategy(n_max, eye, m, prod=numpy.dot):
  """Finds out how to build x**N with low depth, given all the lower powers."""
  depth_and_tensor_power_by_exponent = [None] * (n_max + 1)
  depth_and_tensor_power_by_exponent[0] = (0, eye)
  depth_and_tensor_power_by_exponent[1] = (0, m)
  for n in range(2, n_max + 1):
    best_depth, best_k = min(
        (1 + max(depth_and_tensor_power_by_exponent[k][0],
                 depth_and_tensor_power_by_exponent[n - k][0]),
         k) for k in range(1, n))
    depth_and_tensor_power_by_exponent[n] = (
        best_depth, prod(depth_and_tensor_power_by_exponent[best_k][1],
                         depth_and_tensor_power_by_exponent[n - best_k][1]))
  return depth_and_tensor_power_by_exponent


def _expm_taylor(m, max_pow):
  """Matrix exponentiation via Taylor series."""
  m_id = numpy.eye(m.shape[0])
  powers = _get_taylor_strategy(max_pow, m_id, m)
  fact = 1
  accum = m_id
  for n in range(1, max_pow):
    fact *= n
    accum = accum + powers[n][1] / fact
  return accum


def expm(m, max_taylor_pow=8, max_abs_eigenvalue=0.5):
  """Approximate matrix exponentiation."""
  spectral_radius = max(abs(ev) for ev in numpy.linalg.eigvals(m.astype(float)))
  num_halvings = max(
      0, math.ceil(math.log(spectral_radius / max_abs_eigenvalue, 2)))
  m_small = m * 0.5**num_halvings
  ret = _expm_taylor(m_small, max_taylor_pow)
  for _ in range(num_halvings):
    ret = numpy.dot(ret, ret)
  return ret


def np_load_arrays_from_file(names_and_shapes, filename):
  """Loads a collection of numpy-arrays from a saved vector."""
  with open(filename, 'rb') as h:
    all_data = numpy.load(h)
  offset = 0
  ret = {}
  for name, shape in names_and_shapes:
    a = numpy.zeros(shape)
    a.flat = all_data[offset: offset + a.size]
    offset += a.size
    ret[name] = a
  if offset != all_data.size:
    raise ValueError('Data size mismatch.')
  return ret


def process_example(tensors, ex_img, expm=expm):
  """Processes an example."""
  v_lin_angles = opt_einsum.contract('yxca,yxc->a', tensors['uk'], ex_img)
  v_aff_angles = v_lin_angles + tensors['ub']
  m_lin_gen = opt_einsum.contract('amn,a->mn', tensors['mk'], v_aff_angles)
  m_aff_gen = m_lin_gen + tensors['mb']  # We could also absorb ub into mb.
  m_exp = expm(m_aff_gen)
  v_lin_exp = opt_einsum.contract('mn,mnp->p', m_exp, tensors['sk'])
  v_aff_exp = v_lin_exp + tensors['sb']
  float_weights = v_aff_exp.astype(float)
  float_probabilities = numpy.exp(float_weights) / sum(numpy.exp(float_weights))
  return {k: v for k, v in locals().items()}


def fuzzify(aacontext, array, delta):
  """Fuzzifies a feature-array by adding uncertainty to every parameter."""
  return numpy.array([aacontext.num(x, radius=delta)
                      for x in array.flat]).reshape(array.shape)


def aa_process_example(tensors, ex_img, img_fuzz=1e-5,
                       collapsing_threshold=1e-3,
                       max_num_symbols=10,
                       thorough=False):
  """Processes an example with Affine Arithmetic."""
  aac0 = AAContext(name='aac0',
                   thorough=thorough,
                   collapsing_threshold=collapsing_threshold,
                   max_num_symbols=max_num_symbols)
  fuzzed_img = fuzzify(aac0, ex_img, img_fuzz)
  v_lin_angles = opt_einsum.contract('yxca,yxc->a', tensors['uk'], fuzzed_img)
  aac1 = AAContext(name='aac1',
                   thorough=thorough,
                   collapsing_threshold=collapsing_threshold,
                   max_num_symbols=max_num_symbols)
  v_lin_angles_aa = numpy.array(
      [aac1.num(float(x), x.radius) for x in v_lin_angles])
  v_aff_angles_aa = v_lin_angles + tensors['ub']
  m_lin_gen = opt_einsum.contract('amn,a->mn', tensors['mk'], v_aff_angles_aa)
  for aaz in m_lin_gen.flat:
    aaz._context.collapse_ess(aaz._ess)
  m_aff_gen = m_lin_gen + tensors['mb']  # We could also absorb ub into mb.
  m_exp = expm(m_aff_gen)
  v_lin_exp = opt_einsum.contract('mn,mnp->p', m_exp, tensors['sk'])
  v_aff_exp = v_lin_exp + tensors['sb']
  float_weights = v_aff_exp.astype(float)
  float_probabilities = numpy.exp(float_weights) / sum(numpy.exp(float_weights))
  return {k: v for k, v in locals().items()}


def mspace_transform_tensors(tensors, mright_transform=None):
  """Applies M_right coordinate transformation to tensors."""
  uk = tensors['uk']
  ub = tensors['ub']
  mk = tensors['mk']
  mb = tensors['mb']
  sk = tensors['sk']
  sb = tensors['sb']
  imr_transform = numpy.linalg.inv(mright_transform)
  mk = numpy.einsum('amn,nR,Qm->aQR',
                    mk, mright_transform, imr_transform, optimize='greedy')
  mb = numpy.einsum('mn,nR,Qm->QR',
                    mb, mright_transform, imr_transform, optimize='greedy')
  sk = numpy.einsum('mnp,Rn,mQ->QRp',
                    sk, imr_transform, mright_transform, optimize='greedy')
  return dict(uk=uk, ub=ub, mk=mk, mb=mb, sk=sk, sb=sb)


def determine_robustness_aa(model_tensors,
                            ex_img,
                            Linf_bounds_to_check,
                            coordinate_transform_to_example=False,
                            **aa_kwargs):
  """Determines robustness bounds via AA."""
  if coordinate_transform_to_example:
    processed = process_example(model_tensors, ex_img)
    m_gen = processed['m_aff_gen']
    _, e0g_eigvecsT = numpy.linalg.eig(m_gen)
    model_tensors = mspace_transform_tensors(model_tensors, e0g_eigvecsT)
  for fuzz in sorted(Linf_bounds_to_check, reverse=True):
    aa_processed = aa_process_example(model_tensors, ex_img, img_fuzz=fuzz,
                                      **aa_kwargs)
    v_evidence = aa_processed['v_aff_exp']
    m_margins = [[vj - vk for vk in v_evidence] for vj in v_evidence]
    if any(all(float(v) >= v.radius for v in row) for row in m_margins):
      return fuzz  # For this fuzz, we have guaranteed robustness.
  return 0


def check_robustness_claims():
  """Checks claims about robustness from the paper."""
  (x_train, y_train), (x_test, y_test) = load_cifar10()
  model_tensors = load_the_cifar10_model()
  # Check that the model classifies a small set of examples as expected.
  # Major problems (e.g. having loaded the wrong model file) would
  # show up here.
  probs = [
      tuple(numpy.round(process_example(
          model_tensors,
          x_train[k],
      )['float_probabilities'], 3))
      for k in range(5)]
  expected_probs = (
      [(0.017, 0.035, 0.191, 0.072, 0.032, 0.013, 0.6, 0.026, 0.009, 0.006),
       (0.003, 0.008, 0.123, 0.04, 0.703, 0.017, 0.046, 0.055, 0.001, 0.003),
       (0.037, 0.036, 0.301, 0.041, 0.074, 0.012, 0.453, 0.024, 0.015, 0.009),
       (0.266, 0.239, 0.345, 0.053, 0.001, 0.026, 0.026, 0.014, 0.014, 0.015),
       (0.013, 0.174, 0.094, 0.076, 0.316, 0.034, 0.199, 0.028, 0.045, 0.02)])
  assert probs == expected_probs, \
         'Model did not classify a small sample of examples as expected.'
  #
  test_weights = [
      tuple(process_example(
          model_tensors,
          img,
      )['float_weights'])
      for img in x_test]
  n_weights_of_correct_classifications = [
      (n, weights) for n, (weights, y) in enumerate(zip(test_weights, y_test))
      if y[numpy.argmax(weights)] == 1]
  accuracy = len(n_weights_of_correct_classifications) / len(y_test)
  assert accuracy > 0.52  # Implicit claim: Accuracy of this small model is OK.
  #
  tu = numpy.einsum('yxca,amn->yxcmn',
                    model_tensors['uk'],
                    model_tensors['mk'])
  m = numpy.einsum('yxcmn->mn', abs(tu))
  delta_in = numpy.linalg.svd(m @ m.T)[1][0]**.5
  assert numpy.round(delta_in, 3) == 213.598, \
         'Claim L.376R: delta_in ~ 200'
  s_2_norm = numpy.linalg.svd(model_tensors['sk'].reshape(-1, 10))[1][0]
  assert numpy.round(s_2_norm, 3) == 3.153, 'Claim L.376R, |S|_2 ~ 3'
  ms = numpy.einsum('yxcmn,byxc->bmn', tu, x_test)
  m_2_norms = sorted([max(abs(ev) for ev in numpy.linalg.eigvals(m))
                      for m in ms])
  assert m_2_norms[int(0.98 * len(m_2_norms))] < 4, \
         'Claim L.377R: |M|_2 < 4 for 98% of test cases.'
  # We actually can strengthen this claim.
  top_evidences = [sorted(ws, reverse=True)[:2]
                   for _, ws in n_weights_of_correct_classifications]
  evidence_margins = sorted([first - second for first, second in top_evidences],
                            reverse=True)
  assert evidence_margins[int(0.63 * len(evidence_margins))] > 1, \
         ('Claim L.380R: Evidence margins at least 1 for >63% of '
          'correctly-classified cases.')
  #
  def get_Linf_bound(ex_img):
    """Determines a L_infinity bound."""
    processed = process_example(model_tensors, ex_img)
    m_gen = processed['m_aff_gen']
    m_radius = numpy.linalg.svd(m_gen @ m_gen.T)[1][0]**.5
    sensitivity_factor = (
        m_gen.shape[0]**.5 * math.exp(m_radius) * s_2_norm)
    weights = sorted(processed['float_weights'], reverse=True)
    evidence_margin = 0.5 * (weights[0] - weights[1])
    ret = scipy.special.lambertw(evidence_margin /
                                 sensitivity_factor) / delta_in
    assert abs(ret.imag) < 1e-8
    return ret.real
  #
  linf_bounds = [get_Linf_bound(x_test[n])
                 for n, _ in n_weights_of_correct_classifications]
  plot_loghist(linf_bounds, 50, filename=None)


def check_analytic_formula_aligns_with_aa(num_samples=100):
  """Checks that the analytic formula compares well with AA.

  This function asserts that it is not easy to outperform the
  analytic result on guaranteed robustness with AA.
  """
  (x_train, y_train), (x_test, y_test) = load_cifar10()
  model_tensors = load_the_cifar10_model()
  test_weights = [
      tuple(process_example(
          model_tensors,
          img,
      )['float_weights'])
      for img in x_test]
  n_weights_of_correct_classifications = [
      (n, weights) for n, (weights, y) in enumerate(zip(test_weights, y_test))
      if y[numpy.argmax(weights)] == 1]
  #
  rng = numpy.random.RandomState(seed=0)  # Make reproducible.
  ns_correct = [n for n, _ in n_weights_of_correct_classifications]
  rng.shuffle(ns_correct)
  ns_correct_samples = ns_correct[:num_samples]
  robustnesses = [
      determine_robustness_aa(
          model_tensors, x_test[n],
          [1e-7,  # Skip: 3e-7
           1e-6, 3e-6,
           1e-5, 3e-5,
           1e-4],
          # Using more AA error-symbols has been found to only slightly
          # improve bounds, while having a strong negative impact on
          # runtime.
          max_num_symbols=3, thorough=False)
      for n in ns_correct_samples]
  if num_samples == 100:
    assert (sorted(collections.Counter(robustnesses).items()) ==
            [(0, 1), (1e-07, 3), (1e-06, 13), (3e-06, 32), 
             (1e-05, 47), (3e-05, 4)
             ]), 'AA-Robustnesses guarantees are not as expected.'
  return list(zip(ns_correct, robustnesses))


print('=== Checking Robustness Claims ===')
check_robustness_claims()
print('=== Formula/AA Comparison ===')
check_analytic_formula_aligns_with_aa()
